# **Setup Folders for Collection**

In [1]:
import numpy as np
import os
import cv2 as cv
import mediapipe as mp
from mediapipe.python.solutions.pose import PoseLandmark
import math

In [2]:
# DATA_PATH = os.path.join('Sign_Data_Upgrade')
# DATA_PATH = os.path.join('My_Temp_Datasets')
DATA_PATH = os.path.join('Datasets')

# actions = np.array(['maaf', 'tolong', "terimakasih", "nama", "saya", "kamu", "siapa"])

# untuk testing hanya satu per satu dahulu
# actions = np.array(['tolong', 'maaf', 'nama', 'kamu', 'siapa'])
actions = np.array(['Maju', 'Mundur', 'Kiri', 'Kanan', 'Stop'])

# menggunakan xx video
# no_sequences = 40

# untuk pengambilan data
no_sequences = 30

# temp_no_sequences = 10
# no_sequences = temp_no_sequences


# setiap video berisi xx frame (5 frame transition)
sequence_length = 30

In [3]:
def create_folder(actions, data_path, no_sequences):
    for action in actions: 
        for sequence in range(no_sequences):
            try: 
                os.makedirs(os.path.join(data_path, action, str(sequence)))
            except:
                pass

In [4]:
create_folder(actions, DATA_PATH, no_sequences)
#create_folder(actions, DATA_PATH, temp_no_sequences)

In [5]:
# # FOR CHANGING FOLDER NUMBER
# def change_file_sequence(folder_name, total_sequences, start_next_sequences, actions):
#     DATA_PATH_CHANGE = os.path.join(rf'D:\1 PraTA\Dataset\{folder_name}')

#     next_folder = start_next_sequences

#     for action in actions:
#         for new_sequence in range(total_sequences):
#             PATH_OLD = os.path.join(DATA_PATH_CHANGE, action, str(new_sequence))
#             PATH_NEW = os.path.join(DATA_PATH_CHANGE, action, str(new_sequence + next_folder))

#             print(PATH_OLD + "\n")
#             print(PATH_NEW)
#             print("------------")
    
#             os.rename(PATH_OLD, PATH_NEW)         

In [6]:
# change_file_sequence(DATA_PATH, total_sequences=10, start_next_sequences=20, actions=actions)

# **Collect Keypoint Values for Training and Testing**

In [7]:
#define mediapipe holistic and drawing utils
# mp_holistic = mp.solutions.holistic
# mp_drawing = mp.solutions.drawing_utils
# mp_drawing_styles = mp.solutions.drawing_styles
# mp_pose = mp.solutions.pose
# MediaPipe initialization
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

In [8]:
#detect using mediapipe model
def media_pipe_detection(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB) 
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR) 
    return image, results

In [9]:
# #draw landmarks without style
# def draw_land_marks(image, results):
#     # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    
#     # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

#     custom_pose_connections = list(mp_pose.POSE_CONNECTIONS)
    
#     excluded_landmarks = [
#         PoseLandmark.NOSE,
#         PoseLandmark.LEFT_EYE_INNER,
#         PoseLandmark.LEFT_EYE,
#         PoseLandmark.LEFT_EYE_OUTER,
#         PoseLandmark.RIGHT_EYE_INNER,
#         PoseLandmark.RIGHT_EYE,
#         PoseLandmark.RIGHT_EYE_OUTER,
#         PoseLandmark.LEFT_EAR,
#         PoseLandmark.RIGHT_EAR,
#         PoseLandmark.MOUTH_LEFT,
#         PoseLandmark.MOUTH_RIGHT,
#         PoseLandmark.LEFT_HIP,
#         PoseLandmark.RIGHT_HIP,
#         PoseLandmark.LEFT_KNEE,
#         PoseLandmark.RIGHT_KNEE,
#         PoseLandmark.LEFT_ANKLE,
#         PoseLandmark.RIGHT_ANKLE,
#         PoseLandmark.LEFT_HEEL,
#         PoseLandmark.RIGHT_HEEL,
#         PoseLandmark.LEFT_FOOT_INDEX,
#         PoseLandmark.RIGHT_FOOT_INDEX
#     ]

#     for landmark in excluded_landmarks:
#         custom_pose_connections = [connection_tuple for connection_tuple in custom_pose_connections if landmark.value not in connection_tuple]

#     mp_drawing.draw_landmarks(image, results.pose_landmarks, connections=custom_pose_connections)
#     mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
#     mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

# Draw landmarks without style
def draw_landmarks(image, results):
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw the facial landmarks with specified drawing specs.
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
            )


In [10]:
# def extract_keypoints(results):
#     # face = np.array([[res.x, res.y] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*2)

#     # pose = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*2)

#     if results.pose_landmarks:
#         selected_pose_landmarks = results.pose_landmarks.landmark[11:23]
#         pose = np.array([[res.x, res.y] for res in selected_pose_landmarks]).flatten()
#     else:
#         pose = np.zeros(22*2)

        
#     left_hand = np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*2)
#     right_hand = np.array([[res.x, res.y] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*2)
   
#     # return np.concatenate([pose, face, left_hand, right_hand])
#     return np.concatenate([pose, left_hand, right_hand])

def extract_keypoints(results):
    if results.multi_face_landmarks:
        face = np.array([[res.x, res.y, res.z] for landmark in results.multi_face_landmarks for res in landmark.landmark]).flatten()
    else:
        face = np.zeros(468*3)
    return face


In [11]:
# cap = cv.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

#     for action in actions:
#         for sequence in range(no_sequences):
#             for frame_num in range(sequence_length):

#                 ret, frame = cap.read()

#                 image, results = media_pipe_detection(frame, holistic)
                
#                 #get ready for capture current sequence
#                 if frame_num == 0: 
#                     cv.putText(image, 'STARTING COLLECTION', (120,200), 
#                                cv.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv.LINE_AA)
#                     cv.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv.LINE_AA)
#                     cv.imshow('OpenCV Feed', image)
#                     cv.waitKey(5000)
                
#                 #prepare for next sequence
#                 elif frame_num == sequence_length - 1 :
#                     cv.putText(image, 'NEXT SEQUENCE', (120,200), 
#                                cv.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv.LINE_AA)
#                     cv.imshow('OpenCV Feed', image)
#                     cv.waitKey(5000)

#                 else: 
#                     cv.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv.LINE_AA)
#                     cv.imshow('OpenCV Feed', image)

#                 black_bg = np.zeros((frame.shape[0], frame.shape[1], frame.shape[2]))

#                 draw_land_marks(image, results)
#                 draw_land_marks(black_bg, results)

#                 cv.imwrite(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}-clear.jpg"), frame)
#                 cv.imwrite(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.jpg"), image)
#                 cv.imwrite(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}-black.jpg"), black_bg)
                
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)

#                 if cv.waitKey(10) & 0xFF == ord('q'):
#                     break
                    
#     cap.release()
#     cv.destroyAllWindows()

# Initialize video capture
cap = cv.VideoCapture(0)
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:

    # Define actions, no_sequences, sequence_length, and DATA_PATH here
    actions = ['Maju', 'Mundur', 'Kiri', 'Kanan', 'Stop']  # replace with your actual actions
    no_sequences = 5
    sequence_length = 30
    DATA_PATH = 'D:/1 PraTA/Dataset'  # replace with your actual data path

    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()
                if not ret:
                    print("Ignoring empty camera frame.")
                    continue

                # Convert the BGR image to RGB and process it with MediaPipe Face Mesh.
                image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                results = face_mesh.process(image)

                # Draw the face mesh annotations on the image.
                image.flags.writeable = True
                image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
                draw_landmarks(image, results)  # Make sure you have defined this function

                # Display the image
                cv.imshow('MediaPipe FaceMesh', image)

                # Save keypoints data
                keypoints = extract_keypoints(results)  # Make sure you have defined this function
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

        if cv.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'D:/1 PraTA/Dataset\\Maju\\0\\0.npy'

: 

In [ ]:
cap.release()
cv.destroyAllWindows()

# **Normalize Datasets**

In [ ]:
coor_x = []
coor_y = []
coordinates = []

for action in actions:
     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num + 1)))
            for index, coor in enumerate(res):
                coordinates.append(coor)

                if(index % 2 != 0):
                    coor_x.append(coor)
                else:
                    coor_y.append(coor)

In [ ]:
print(len(coordinates))

194400


In [ ]:
midpoint_x = (coor_x[0] + coor_x[1])/2
midpoint_y = (coor_y[0] + coor_y[1])/2

# delta_x = midpoint_x - frame_width/2
# delta_y = midpoint_y - frame_height/2

shoulder_delta_x = coor_x[1] - coor_x[0]
shoulder_delta_y = coor_y[1] - coor_y[0]
shoulder_width = math.sqrt(pow(shoulder_delta_x, 2) + pow(shoulder_delta_y, 2))

In [ ]:
def normalization(midpoint_x, midpoint_y, shoulder_width, coordinates):
    print("Koordinat awal:")
    print("Max: " + str(max(coordinates)))
    print("Min: " + str(min(coordinates)))


    for index, coor in enumerate(coordinates):
        
        #position invariant
        if(index % 2 != 0):
            coordinates[index] -= midpoint_x
        else:
            coordinates[index] -= midpoint_y

        #scale invariant
        coordinates[index] /= shoulder_width

    print("\nKoordinat normalisasi:")
    print("Max: " + str(max(coordinates)))
    print("Min: " + str(min(coordinates)))
    return coordinates

def save_normalization(paths, actions, no_sequences, sequence_length, coor_norm):
    total_landmarks = 108
    temp_index = 0

    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                saved_coor_norm = coor_norm[temp_index:temp_index+total_landmarks]

                norm_npy_path = os.path.join(paths, action, str(sequence), str(frame_num + 1) + "-norm")
                np.save(norm_npy_path, saved_coor_norm)

                temp_index += total_landmarks

In [ ]:
normalized_coor = normalization(midpoint_x, midpoint_y, shoulder_width, coordinates)

save_normalization(DATA_PATH, actions, no_sequences, sequence_length, normalized_coor)

Koordinat awal:
Max: 1.0613200664520264
Min: 0.0

Koordinat normalisasi:
Max: 3.938435094378623
Min: -2.9151389299256523


In [ ]:
check_coordinates = []

for action in actions:
     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}-norm.npy".format(frame_num+1)))
            for index, coor in enumerate(res):
                check_coordinates.append(coor)

In [ ]:
print(len(check_coordinates))
print("Max: " + str(max(check_coordinates)))
print("Min: " + str(min(check_coordinates)))

194400
Max: 3.938435094378623
Min: -2.9151389299256523


In [ ]:
# print(len(coor_x) * 2)
# print(coor_x[0])
# print(2*30*40*108)
# print(12*2 + 21 * 2 * 2)

259200
0.44267737865448
259200
108


In [ ]:
108*30*30*2

194400